In [1]:
import numpy as np
import re
import random

import tensorflow as tf
import sklearn as sk

import Functions as fn
from DS import DS
from Set import pool
from Iterator import Iterator
from FFModel import FF_Model

# Data Loading

In [2]:
#Dataset = fn.first_time_load()
#Dataset.write_texts()
#Dataset.write_labels()

Dataset = pool()
Dataset.load_texts()
Dataset.load_labels()

Texts loaded
Labels loaded


In [3]:
#print('Number of Texts: ', Dataset.size)
#print('Number of 2007 Smoking Challenge texts: ', Dataset.number_of(challenge='2007 Smoking Challenge'))
#print('Number of 2008 ObesityChallenge texts: ', Dataset.number_of(challenge='2008 Obesity Challenge'))
#print('Number of 2009 Medication Challenge texts: ', Dataset.number_of(challenge='2009 Medication Challenge'))
#print('Number of 2010 Relations Challenge texts: ', Dataset.number_of(challenge='2010 Relations Challenge'))
#print('Number of 2011 Coreference Challenge texts: ', Dataset.number_of(challenge='2011 Coreference Challenge'))
#print('Number of 2012 Temporal Relations Challenge texts: ', Dataset.number_of(challenge='2012 Temporal Relations Challenge'))
#print('Number of Train Texts: ', Dataset.number_of(stage='train'))
#print('Number of Test Texts: ', Dataset.number_of(stage='test'))
#print('Number of Labeled Texts: ', Dataset.number_of(labelled='yes'))
#print('Number of Initially Labeled Texts: ', Dataset.number_of(labelled='yes', label_type='train'))
#print('Number of Competitor Labeled Texts Texts: ', Dataset.number_of(labelled='yes', label_type='test'))

# Embedding Generation

In [4]:
#Dataset.process_for_embedding()
#sentences = Dataset.get_sentences()

In [5]:
from gensim.models import Word2Vec
#model = Word2Vec(sentences, min_count=1, size=100)
model = Word2Vec.load('W2V')

#fn.write_emb('testemb', model)
#model = fn.load_emb('testemb')
#model.save('W2V')

C:\Users\Luka\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
#model.most_similar("prozac", topn=10)

# Visualisation

In [7]:
targets = ['medications', 'dosages', 'modes', 'frequencies', 'durations', 'reasons']
target_dict = {label: words for (label, words) in zip(targets, list(fn.label_words(Dataset, model)))}

Number of: m=1327, do=190, mo=104, f=227, du=178, r=715


In [8]:
#fn.visualise(model, sentences, [target_dict['medications']], 1000, 'Medications in top 1000 words')

In [9]:
#print_set = []
#for case in Dataset.get_DS(labelled='yes').data:
#    for term in re.finditer(r'm="[^|]+\|', case.raw_labels):
#        print_set.append(term.group()[:-1])
#print(*print_set, sep='\n')

# Naive Test 

In [10]:
#for target in targets:
#    for reps in [True, False]:
#        target_size = len(target_dict[target])
#        sets = fn.get_traintest(model, target_dict[target], target_size * 5, target_size, 10, 50, reps, reps)
#        sets = {'train_set': sets[0], 'train_labels': sets[1], 'validation_set': sets[2], 'validation_labels': sets[3], 'test_set': sets[2], 'test_labels': sets[3]}
#        print("Target: %s \tRepetitions: %s" % (target, reps))
#        NN = FF_Model()
#        NN.build_graph()
#        NN.train(sets, report_percentage=10)

# FFNN Testing 

In [ ]:
labelled_cases = Dataset.get_DS(labelled='yes')
labelled_cases.process_for_testing()

In [ ]:
#emb_sizes = [100]
#emb_models = 5
#target_saturations = [0.1]
#layer_sizes = [100]
#dropouts = [1.0]
#learn_rates = [0.01]
#epochs = [100]
#NN_num = 5

#case_num = len(emb_sizes)*emb_models*len(layer_sizes)*len(target_saturations)*len(epochs)*len(dropouts)*len(learn_rates)*NN_num
#print(case_num)

In [ ]:
max_performance = 0
n = 1

for emb_size in emb_sizes:
    print('Model Number: %d/%d' %(n, case_num))
    for i in range(emb_models):
        model = Word2Vec(sentences, min_count=1, size=emb_size)
        for saturation in target_saturations:
            sets = fn.get_traintest2 (labelled_cases, model)
            fn.saturate_training_set(sets, model, target_dict['medications'], saturation)
            for layer_size in layer_sizes:
                for drop in dropouts:
                    for rate in learn_rates:
                        for epoch in epochs:
                            for j in range(NN_num):
                                print('Model Number: %d/%d' %(n, case_num))
                                print('ES: %d EM: %d sat: %f, LS: %d, drop: %f, LR: %f, epochs: %d, NN: %d' \
                                       % (emb_size, i, saturation, layer_size, drop, rate, epoch, j))
                                NN = FF_Model(input_size=emb_size, layers=[layer_size], dropout=drop, learn_rate=rate)
                                NN.build_graph()
                                NN.train(sets, epochs=epoch)
                                res = NN.predict(sets['test_set'])
                                tru = np.argmax(sets['test_labels'], 1)
                                perf = sk.metrics.f1_score(tru, res, pos_label=0)
                                if perf > max_performance:
                                    max_performance = perf
                                    NN.save_model('gold')
                                    model.save('gold/GOLDEMB')
                                NN.close()
                                n += 1

# Play 

In [ ]:
sets = fn.get_traintest2(labelled_cases, model)

In [ ]:
print(np.array(sets['train_labels']).sum(0)/ len(sets['train_labels']))

In [ ]:
fn.saturate_training_set(sets, model, target_dict['medications'], 0.1)

In [ ]:
res = NN.predict(sets['test_set'])
tru = np.argmax(sets['test_labels'], 1)

In [ ]:
tp = len([a for a in range(len(tru)) if (res[a] == 0) and (tru[a] == 0)])
tn = len([a for a in range(len(tru)) if (res[a] == 1) and (tru[a] == 1)])
fp = len([a for a in range(len(tru)) if (res[a] == 0) and (tru[a] == 1)])
fn = len([a for a in range(len(tru)) if (res[a] == 1) and (tru[a] == 0)])
tp, tn, fp, fn

In [ ]:
sk.metrics.f1_score(tru, res, pos_label=0, average='binary')

In [ ]:
[(sets['test_words'][a], res[a], tru[a]) for a in range(len(sets['test_words'])) if res[a] == 1 and tru[a] == 0]

In [ ]:
proba = []
for i in range(len(res)):
    if not (sets['test_words'][i] == ')' or sets['test_words'][i] == '('):
        proba.append([sets['test_words'][i], res[i], tru[i]])

In [ ]:
sk.metrics.f1_score(np.array(proba)[:,2].astype(int), np.array(proba)[:,1].astype(int), pos_label=0, average='binary')

In [ ]:
len([(proba[a][0], proba[a][1], proba[a][2]) for a in range(len(proba)) if proba[a][1] == 1 and proba[a][2] == 0])

In [ ]:
[(proba[a][0], proba[a][1], proba[a][2]) for a in range(len(proba)) if proba[a][1] == 1 and proba[a][2] == 0]

In [ ]:
'projectile' in target_dict['medications']